In [ ]:
import pandas as pd
import geopandas as gpd
import folium


In [ ]:
income_df = pd.read_csv("../data/income/2021_income.csv")
income_df

In [ ]:
# sf stands for shape file
sf = gpd.read_file("../data/shapefile/POA_2016_AUST.shp")

sf['POA_CODE16'] = sf['POA_CODE16'].astype(int)
sf

In [ ]:
# Merge the two data frame
income_df = income_df \
    .merge(sf[['POA_CODE16', 'geometry']], left_on='POA_CODE_2021', right_on='POA_CODE16') \
    .drop('POA_CODE16', axis=1)

income_df

In [ ]:
geoJSON = sf[['POA_CODE16', 'geometry']].drop_duplicates('POA_CODE16').to_json()

In [ ]:
# Map of score

m = folium.Map(tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on more information on how to plot aggregated data.
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=income_df, # data source
    columns=['POA_CODE_2021','Median_tot_fam_inc_weekly'], # the columns required
    key_on='properties.POA_CODE16', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='Median Total Family Income Weekly AUD$'
)

c.add_to(m)

m.save('../plots/income_weekly.html')
m

## Below is the convertion of SA2 to Postcode if needed (example)

In [ ]:

income_df = pd.read_csv("../data/income/SA2_income_2018.csv")
income_df['SA2'] = income_df['SA2'].astype(str)
income_df

In [ ]:
# Read the postcode dataframe and simplify into a dataframe only contains postcode and SA2
postcode_df = pd.read_csv("../data/tables/australian_postcodes.csv")
postcode_df['SA2_MAINCODE_2016'] = postcode_df['SA2_MAINCODE_2016'].fillna(0).astype(int)
convert_df = postcode_df[['postcode','SA2_MAINCODE_2016']]
convert_df = convert_df.rename(columns={'SA2_MAINCODE_2016':'SA2'})
convert_df = convert_df.astype(str)
convert_df = convert_df.drop_duplicates()
convert_df

In [ ]:
merged_df = pd.merge(income_df, convert_df, on='SA2')
merged_df